In [1]:
import numpy as np
import scipy.sparse as sp

import sys
sys.path.append('../')

from best.mdp import MDP, ProductMDP, ParallelMDP

### Define two label MDPs

In [2]:
T0 = np.eye(3)
T1 = np.array([[0,0.5,0.5], [0,1,0], [0,0,1]])

def output_fcn(n):
    if n == 0:
        return 'init'    # label unknown
    if n == 1:
        return 'safe'    # can traverse region
    if n == 2:
        return 'unsafe'  # can not traverse region
    
map1 = MDP([T0, T1], input_name='meas1',
                     output_fcn=output_fcn, output_name='label1')

map2 = MDP([T0, T1], input_name='meas2',
                     output_fcn=output_fcn, output_name='label2')

map_mdp = ParallelMDP([map1, map2])
print map_mdp

MDP: 4 inputs "(meas1, meas2)" --> 9 states "(label1, label2)"


### Define gridworld system MDP

In [3]:
# gridworld mdp
l_x = 4  # x length
l_y = 2  # y length

def n_to_ij(n, l_x):
    return (n%l_x, n//l_x)

def ij_to_n(i,j,l_x):
    return i + j*l_x

T_start = [ij_to_n(i, j, l_x) for i in range(l_x) for j in range(l_y)]

# north
Tn_end =   [ij_to_n(i, max(0, j-1), l_x) for i in range(l_x) for j in range(l_y)]
# south
Ts_end =   [ij_to_n(i, min(l_y-1, j+1), l_x) for i in range(l_x) for j in range(l_y)]
# east
Te_end =   [ij_to_n(min(l_x-1, i+1), j, l_x) for i in range(l_x) for j in range(l_y)]
# west
Tw_end =   [ij_to_n(max(0, i-1), j, l_x) for i in range(l_x) for j in range(l_y)]

Tn = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Tn_end)), shape=(l_x*l_y, l_x*l_y))
Ts = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Ts_end)), shape=(l_x*l_y, l_x*l_y))
Te = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Te_end)), shape=(l_x*l_y, l_x*l_y))
Tw = sp.coo_matrix((np.ones(l_x*l_y), (T_start, Tw_end)), shape=(l_x*l_y, l_x*l_y))

def syst_input_fcn(m):
    if m == 'n':
        return 0
    elif m == 's':
        return 1
    elif m == 'e':
        return 2
    else:
        return 3

syst_mdp = MDP([Tn, Ts, Te, Tw], input_fcn=syst_input_fcn, input_name='dir', output_name='s')
print syst_mdp

MDP: 4 inputs "dir" --> 8 states "s"


### Connect system and labels

In [4]:
# define mapping s -> (meas1, meas2)
def map_connection(s):
    ret = [0,0]
    if s == 1:
        ret[0] = 1  # label1 is resolved at state 1
    if s == 5:
        ret[1] = 1  # label2 is resolved at state 5
    return set([tuple(ret)])

prod_mdp = ProductMDP(syst_mdp, map_mdp, map_connection)
print prod_mdp

MDP: 4 inputs "dir" --> 72 states "(s, (label1, label2))"


### Solve LTL problem on product system

In [5]:
from best.ltl import solve_ltl_cosafe

formula = '( ( ( !reg1 ) | safe1 ) & ( ( !reg2 ) | safe2 ) ) U target'

# define mapping (s, (label1, label2)) -> 2^2^AP
def dfs_connection(s_label):
    s = s_label[0]
    l1 = s_label[1][0]
    l2 = s_label[1][1]
    ret = []
    if s == 7:   # we want to reach state 7
        ret.append('target')
    if s == 2:   # state 2 is labeled by label1
        ret.append('reg1')
    if s == 6:   # state 6 is labeled by label2
        ret.append('reg2')
    if l1 == 'safe':
        ret.append('safe1')
    if l2 == 'safe':
        ret.append('safe2')
    return set( (tuple(ret),) )

policy = solve_ltl_cosafe(prod_mdp, formula, dfs_connection, algorithm='petter')
print policy.V

iteration 0, time 6.91413879395e-06
iteration 1, time 0.0186839103699
iteration 2, time 0.0193748474121
iteration 3, time 0.0199139118195
iteration 4, time 0.0204989910126
iteration 5, time 0.0210349559784
finished after 5 iterations and 0.0215978622437s
[[ 0.75  1.    0.5   1.    1.    1.    0.5   1.    0.    0.75  1.    0.5
   1.    1.    1.    0.5   1.    0.    1.    1.    1.    1.    1.    1.    1.
   1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    0.75
   1.    0.5   1.    1.    1.    0.5   1.    0.    0.75  1.    0.5   1.    1.
   1.    0.5   1.    0.    1.    1.    1.    1.    1.    1.    1.    1.    1.
   1.    1.    1.    1.    1.    1.    1.    1.    1.  ]
 [ 1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
   1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
   1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.
   1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.    1.  

### Simple simulation (p=0.75 of success)

In [14]:
N = prod_mdp.N  # total number of states
np.random.seed(4)  # fail
np.random.seed(6)  # long path

s_loc = (4, (0,0))
s = prod_mdp.global_state(s_loc)
p = 1

policy.reset()

while not policy.finished() and p > 0:
    print "current state ", prod_mdp.output(s)
  
    # get input
    u, p = policy.get_input(s)
    print "input ", u, "probability of sat ", p

    # update state
    s_vec = np.zeros((N,1))
    s_vec[s] = 1  # mdp vector state
    s_prob = prod_mdp.evolve(s_vec, u)
    s = np.random.choice(range(N), 1, p=s_prob.flatten())[0]  # resolve stochasticity
    
    print "moved to ", prod_mdp.output(s)
    
    # update DFS
    print "reporting ", list(dfs_connection(prod_mdp.output(s)))[0]
    policy.report_aps(list(dfs_connection(prod_mdp.output(s)))[0])
    
    print "\n"

current state  (4, ('init', 'init'))
input  2 probability of sat  0.75
moved to  (5, ('init', 'unsafe'))
reporting  ()


current state  (5, ('init', 'unsafe'))
input  0 probability of sat  0.5
moved to  (1, ('safe', 'unsafe'))
reporting  ('safe1',)


current state  (1, ('safe', 'unsafe'))
input  2 probability of sat  1.0
moved to  (2, ('safe', 'unsafe'))
reporting  ('reg1', 'safe1')


current state  (2, ('safe', 'unsafe'))
input  2 probability of sat  1.0
moved to  (3, ('safe', 'unsafe'))
reporting  ('safe1',)


current state  (3, ('safe', 'unsafe'))
input  1 probability of sat  1.0
moved to  (7, ('safe', 'unsafe'))
reporting  ('target', 'safe1')


